# Listes et récursivité terminale

**Q1** Écrire une fonction `maxl` calculant le maximum d'une liste d'entiers **non vide**.

In [3]:
let rec maxl (l : int list) : int =
    match l with
    | t::[] -> t
    | t::q -> let m = maxl q in if t > m then t else m
    | _ -> 0

val maxl : int list -> int = <fun>

**Q2** Exécutez les cellules suivantes. Pourquoi le second test échoue ?

In [4]:
let test_maxl n = 
    maxl (List.init n (fun _ -> Random.int 0x0fffffff))

val test_maxl : int -> int = <fun>

In [5]:
test_maxl 1000

- : int = 267491446

In [6]:
test_maxl 100000

Stack overflow during evaluation (looping recursion?).

Beaucoup trop de récursion pour trouver un maximum

Pour ne pas avoir ce problème, on va utiliser la récursivité terminale. Cela consiste à avoir un paramètre supplémentaire servant d'accumulateur pour effectuer le calcul et faire en sorte que les appels récursifs ne soient pas nécessaires pour continuer l'exécution de la fonction qui les appelle.

Pour cela, chaque appel récursif doit être une feuille, un nœud **terminal**, de l'arbre d'expression. Par exemple, `1 + f x` n'est pas terminal mais dans `if test then f x else f (x+1)` tous les appels sont terminaux.

Ainsi, la fonction 
```ocaml
let rec somme l acc =
    match l with
    | [] -> acc
    | t::q -> somme q (acc + t)
```
est récursive terminale.

**Q3** Rappeler pourquoi cela résout le problème de la fonction précédente.

On aura a maxium 1 appel récursif sur le stack d'appels

Pour pouvoir l'utiliser, il faut passer une valeur à l'accumulateur qui correspond au cas de base `[]`, donc ici : `somme [1;2;3] 0`.

En pratique, redonner cette valeur `0` à chaque appel n'est pas robuste : on risque de donner une autre valeur par erreur, ou si la sémantique de la fonction change, il faut reprendre chacun des appels.

On définit alors la fonction récursive terminale comme étant une fonction auxillaire et on définit une nouvelle fonction **non récursive** qui se charge d'initialiser les paramètres.

Dans l'exemple précédent, on pourrait écrire

```ocaml
let rec somme_aux l acc =
    match l with
    | [] -> acc
    | t::q -> somme_aux q (t + acc)
   
let somme l = somme_aux l 0
```

On peut également **cacher** la fonction auxiliaire dans le corps de la fonction `somme` avec une déclaration locale :

```ocaml
let somme l =
    let rec somme_aux l acc = 
        match l with
        | [] -> acc
        | t::q -> somme_aux q (t + acc)
    in
        somme_aux l 0
```

Dans la suite, quand on demande de programmer une fonction récursive terminale, il s'agira ainsi d'une fonction ayant cette structure (avec ou sans déclaration locale).

**Q4** Écrire une fonction récursive terminale `différence` telle que `difference [v1;...;vn]` calcule $v_1-v_2 + v_3 - v_4 + ... + (-1)^{n+1} v_n$.

In [5]:
let difference (l : int list) =
    let rec diff l acc epsilon = 
        match l with
        | [] -> acc
        | t::q -> diff q (acc + t*epsilon) (-epsilon)
    in
        diff l 0 (1)

val difference : int list -> int = <fun>

**Prof** : ici il faut mettre des parenthèses autour du `-1`, parce que sinon, `OCaml` lit `(diff l 0) - 1` et pas ce que vous voulez.

In [6]:
assert (difference [5;2;1] = 4);
assert (difference [5;2;1;2] = 2)

- : unit = ()

**Q5** Écrire une fonction `maxl_term` qui calcule le maximum avec une fonction récursive terminale. Attention, ici, la valeur initiale de l'accumulateur va dépendre de la suite.

In [16]:
let rec maxl_term (l : int list) =
    let rec maxl acc l =
        match l with
        | [] -> acc
        | t::q -> if t>acc then maxl t q else maxl acc q
    in 
        maxl (match l with t::q -> t | _-> int_of_float infinity) l 

val maxl_term : int list -> int = <fun>

In [41]:
maxl_term (List.init 100000 (fun _ -> Random.int 0x0fffffff))

- : int = 268428510

# Arbres et continuations

On va considérer ici le type des arbres binaires non étiquettés suivant :

In [44]:
type arbre = E | N of arbre * arbre

type arbre = E | N of arbre * arbre

On rappelle que la hauteur d'un tel arbre $h(t)$ est définie par induction ainsi :

* $h(E) = 0$
* $h(N(l,r)) = 1 + max(h(l), h(r))$.

**Q6** En déduire une fonction `hauteur` qui calcule cette valeur.

In [48]:
let rec hauteur (t : arbre) : int =
    match t with
    | E -> 0
    | N (l,r) -> 1 + max (hauteur r) (hauteur l)

val hauteur : arbre -> int = <fun>

**Q7** Écrire une fonction récursive terminale `peigne_gauche` telle que :

* `peigne_gauche 0 = E`
* `peigne_gauche 1 = N(E,E)`
* `peigne_gauche 2 = N(N(E,E),E)`
* `peigne_gauche 3 = N(N(N(E,E),E),E)`
* ...

In [56]:
let peigne_gauche n =
    let rec peing acc n =
        if n=0 then acc
        else peing (N(acc,E)) (n-1) 
    in
        peing E n

val peigne_gauche : int -> arbre = <fun>

Comme la fonction est récursive terminale, on peut calculer l'arbre suivant :

In [57]:
let t = peigne_gauche 100000

val t : arbre =
  N
   (N
     (N
       (N
         (N
           (N
             (N
               (N
                 (N
                   (N
                     (N
                       (N
                         (N
                           (N
                             (N
                               (N
                                 (N
                                   (N
                                     (N
                                       (N
                                         (N
                                           (N
                                             (N
                                               (N
                                                 (N
                                                   (N
                                                     (N
                                                       (N
                                                         (N
                                                        

Mais comme la fonction `hauteur` n'est pas récursive terminale, on ne peut calculer sa hauteur ainsi :

In [58]:
hauteur t

Stack overflow during evaluation (looping recursion?).

Il n'est simple de faire une fonction récursive terminale pour calculer la hauteur car il y a un travail à faire dans le sous-arbre gauche puis un travail à faire dans le sous-arbre droit. Le premier appel est donc forcément non terminal.

Il existe pourtant une technique classique permettant de rendre de telles fonctions récursives terminales : le passage de **continuations**.

Au lieu de rajouter un accumulateur, on va rajouter un argument fonctionnel `suite` qui indiquera à la fonction qu'on appelle ce qu'on veut qu'elle fasse avec le résultat.

Par exemple, si on considère `fact` :
```ocaml
let rec fact n =
    if n = 0
    then 1
    else n * fact (n-1)
```

On peut rajouter un paramètre ainsi `suite` et la nouvelle fonction calculera `suite(n!)` :

```ocaml
let rec fact_aux n suite =
    if n = 0
    then suite 1
    else fact_aux (n-1) (fun v -> suite (n * v))
```

Pour récupérer `fact`, il suffit d'appliquer l'identité :

```ocaml
let fact n = fact_aux n (fun v -> v)
```

La fonction est ainsi une sorte d'accumulateur fonctionnel où on retarde le moment où on va effectuer le calcul. Si on considère `fact 4` avec la fonction de départ, on va effectuer dans l'ordre les calculs suivants en remontant :

```
1
1 * 1 = 1
2 * 1 = 2
3 * 2 = 6
4 * 6 = 24
```

Avec une version récursive terminale, on effectuerait les calculs en descendant :

```
1
4 * 1 = 4
3 * 4 = 12
2 * 12 = 24
1 * 24 = 24
```

Ici, l'appel final `suite 1` dans la version avec continuation reviendra à évaluer l'expression suivante :
```ocaml
(fun a -> 4 *
    ((fun b -> 3 *
        ((fun c -> 2 *
            ((fun d -> 1 *
                ((fun e -> e) d))
              c))
          b))
      a)) 1
```
       
Pour l'évaluer, on va propager chaque argument depuis l'appel le plus extérieur :

```ocaml
4 * (fun b -> 3 *
        ((fun c -> 2 *
            ((fun d -> 1 *
                ((fun e -> e) d))
              c))
          b))
      1)
4 * (3 * (fun c -> 2 *
        ((fun d -> 1 *
            ((fun e -> e) d))
          c))
      1)
4 * (3 * (2 * (fun d -> 1 *
            ((fun e -> e) d))
          1))
4 * (3 * (2 * (1 * ((fun e -> e) 1))))
4 * (3 * (2 * (1 * 1)))
```

**Conclusion** : on a effectué les calculs dans l'ordre de l'appel récursif naïf mais avec une fonction récursive terminale !

On rappelle la fonction `map` naïve non récursive terminale et sa version récursive terminale qui effectue un renversement en raison de l'accumulateur.

In [ ]:
let rec map f l =
    match l with
    | [] -> []
    | t::q -> f t :: map f q

In [ ]:
let map_rev f l =
    let rec map_rev_aux f l acc =
        match l with
        | [] -> acc
        | t::q -> map_rev_aux f q (f t :: acc)
    in
        map_rev_aux f l []

**Q8** Écrire une fonction `map_cont` récursive terminale à l'aide de continuations et qui ne renverse donc pas l'ordre.

In [ ]:
let map_cont f l =
    let rec map_count_aux f l suite =
        match l with
        | [] -> suite []
        | t::q -> map_count_aux f t :: (fun -> suite ) 
        



    if n = 0
    then suite 1
    else fact_aux (n-1) (fun v -> suite (n * v))

Si on exécute les cellules suivantes, on peut voir que la version avec continuation fonctionne comme la version `map_rev` mais à l'endroit.

**ATTENTION** une limitation de l'implémentation d'OCaml en JavaScript empêche de se rendre compte que la fonction `map_cont_aux` est récursive terminale. Donc, ici, cela va faire un `stack overflow` mais pas sur la vraie implémentation d'OCaml.

In [ ]:
let _ = map_rev ((+)1) (List.init 100000 (fun i -> i)) in ()

In [ ]:
let _ = map_cont ((+)1) (List.init 100000 (fun i -> i)) in ()

**Q9** Comparez l'espace mémoire utilisé par `map_cont`, `map` et `map_rev`. Que peut-on en déduire ? La pile a-t-elle vraiment disparue dans `map_cont` ?

*Réponse*

Voici une implémentation de `hauteur` utilisant des continuations.

**Q10** Décrire la suite des appels effectués quand on appelle `hauteur (N(E,N(E,E)))`.

*Réponse*

In [ ]:
let hauteur t =
    let rec aux t suite =
        match t with
        | E -> suite 0
        | N(g,d) -> aux g (fun hg ->
            aux d (fun hd -> suite (1 + max hg hd)))
    in
        aux t (fun h -> h)

In [ ]:
hauteur (N(E,N(E,E)))

In [ ]:
hauteur t (* Attention stack overflow 
             dans le navigateur mais pas en vrai *)

**Q11** Écrire une fonction `taille` sur le même modèle qui renvoie la taille (le nombre de nœuds `N` d'un arbre).

In [ ]:
let taille t =
    ...

# Défonctionnalisation (MPI❄️ uniquement)

L'usage d'une fonction pour la continuation ne semble pas être absolument nécessaire quand on n'y effectue que des opérations très simples.

Par exemple, pour `map`, la continuation est soit la fonction identité, soit le cons de `f x` sur une liste suivi du reste de la continuation. On peut ainsi *réifier* la continuation en remplaçant la fonction par une valeur d'un type *ad hoc* :

In [ ]:
type 'a cont = Identite | ConsCont of 'a * 'a cont

In [ ]:
let rec applique cont l =
    match cont with
    | Identite -> l
    | ConsCont(x, cont') ->
        applique cont' (x :: l)

In [ ]:
let map_cont2 f l =
    let rec map_cont2_aux f l cont =
        match l with
        | [] -> applique cont l
        | t::q -> map_cont2_aux f q
                (ConsCont (f t, cont))
    in
        map_cont2_aux f l Identite

Petit bonus de cette version, l'absence de fonction permet la détection de la récursivité terminale avec l'implementation du navigateur. Ainsi, le code suivant fonctionne :

In [ ]:
let _ = map_cont2 ((+)1) (List.init 100000 (fun i -> i)) in ()

En fait, on vient exactement de retrouver le code qu'on connaissait déjà en effecutant deux passages pour renverser la liste.

```ocaml
let rec reverse l acc =
    match l with
    | [] -> acc
    | x :: l' -> reverse l' (x :: acc)
    
let map f l =
    let rec map_aux f l acc =
        match l with
        | [] -> reverse acc []
        | t::q -> map_aux f q (f t :: acc)
    in
        map_aux f l []
```
On pourrait trouver ça décevant, mais l'idée est d'utiliser la relative facilité d'écriture de la version par continuations et la défonctionnaliser pour en déduire une version récursive terminale simple qu'on n'aurait pas pu écrire seul.

On va présenter ici comment défonctionnaliser `hauteur` : on a vu que la continuation est ici de trois formes :

* l'identité
* le fait de relancer un calcul à droite connaissant la valeur à gauche
* le fait d'agréger les calculs à gauche et à droite

In [ ]:
type cont =
    | Identite
    | RelanceDroite of arbre * cont
    | CalculFait of int * cont

In [ ]:
let hauteur t =
    let rec aux t suite =
        match t with
        | E -> applique suite 0
        | N(g,d) -> aux g (RelanceDroite(d, suite))
    and applique suite v =
         match suite with
         | Identite -> v
         | RelanceDroite(t, suite') ->
             aux t (CalculFait(v, suite'))
         | CalculFait(v', suite') -> applique suite' (1 + max v v')
    in
        aux t Identite

In [ ]:
hauteur t

Si on compare avec la fonction `hauteur` initiale, la version récursive terminale obtenue était difficilement envisageable.

**Q12** Que pensez-vous de cette technique pour écrire des fonctions en `C` ?

*Réponse*

**Q13** Quel est la complexité en espace de cette nouvelle fonction `hauteur` ?

*Réponse*

**Q14** Appliquer ce qu'on vient de faire pour les trois fonctions suivantes. A chaque fois, il s'agit de les écrire avec des continuations puis de défonctionnaliser.

In [ ]:
let rec separe l =
    match l with
    | [] -> [], []
    | t::q -> let l1, l2 = separe q in l2, t::l1

let rec fusion l1 l2 =
    match l1, l2 with
    | [], _ -> l2
    | _, [] -> l1
    | t1::q1, t2::q2 ->
        if t1 <= t2
        then t1 :: fusion q1 l2
        else t2 :: fusion l1 q2

let rec tri_fusion l =
    match l with
    | [] | [_] -> l
    | _ -> let l1, l2 = separe l in
        fusion (tri_fusion l1) (tri_fusion l2)

In [ ]:
(* A vous *)